In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
import re
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df= pd.read_csv("/content/tennis.csv")
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [ ]:
import random
i= random.randint(0, len(df))
df['article_text'][i]

"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours with

In [ ]:
from nltk.tokenize import sent_tokenize
sentences= []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))
sentences= [y for x in sentences for y in x]

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-01-30 06:43:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-01-30 06:43:18--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-01-30 06:43:19--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
word_embeddings= {}
f= open('glove.6B.300d.txt', encoding='utf-8')
for line in f:
  values= line.split()
  word= values[0]
  coefs= np.asarray(values[1:], dtype='float32')
  word_embeddings[word]= coefs
f.close()

clean_sentences= pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences= [s.lower() for s in clean_sentences]
from nltk.corpus import stopwords
stop_words= stopwords.words('english')
def remove_stopwords(sen):
  sen_new= " ".join([i for i in sen if i not in stop_words])
  return sen_new
clean_sentences= [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
sentence_vectors= []
for i in clean_sentences:
  if len(i)!= 0:
    v= sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v= np.zeros((300,))
  sentence_vectors.append(v)

In [ ]:
sim_mat= np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i!= j:
      sim_mat[i][j]= cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [ ]:
import networkx as nx
nx_graph= nx.from_numpy_array(sim_mat)
scores= nx.pagerank(nx_graph)

In [ ]:
ranked_sentences= sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse= True)
from termcolor import colored
i= random.randint(0, len(df))
print(colored(('ARTICLE:'.center(50)), 'yellow'))
print('\n')
print(colored((df['article_text'][i]), 'blue'))
print('\n')
print(colored(('SUMMARY:'.center(50)), 'green'))
print('\n')
print(colored((ranked_sentences[i][1]), 'cyan'))

                     ARTICLE:                     


Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, 